In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
from tqdm import tqdm
import json
import cv2
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

CT

In [ ]:
class_list=['0100.Ischemic Stroke','0200.Hemorrhagic Stroke','0300.Stroke Control Group']
save_class_list=['Ischemic','Hemorrhagic','Normal']
data_dir='../../data/stroke_2023/**/1.원천데이터/'
save_dir='../../data/stroke_CT/'

In [ ]:
for j in range(len(class_list)):
    data_list=glob(data_dir+class_list[j]+'/*NCCT/*NCCT*.dcm')
    for i in tqdm(range(len(data_list))):
        folder_split=os.path.basename(data_list[i]).split('_')
        folder_name=folder_split[0]+'_'+folder_split[1]
        folder_path=save_dir+save_class_list[j]+'/'+folder_name+'/'
        create_dir(folder_path)
        dicom_file=data_list[i]
        dcm = pydicom.dcmread(dicom_file)
        img = dcm.pixel_array.astype(np.float32)
        # HU 변환 적용
        slope = dcm.RescaleSlope if 'RescaleSlope' in dcm else 1
        intercept = dcm.RescaleIntercept if 'RescaleIntercept' in dcm else 0
        img = img * slope + intercept  # HU 변환
        # DICOM 헤더에서 Window Level과 Width 가져오기
        window_center = dcm.WindowCenter if hasattr(dcm, 'WindowCenter') else None
        window_width = dcm.WindowWidth if hasattr(dcm, 'WindowWidth') else None

        # Window Level/Width가 여러 개일 경우 첫 번째 값 사용
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = window_center[0]
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = window_width[0]

        if window_center is not None and window_width is not None:
            min_val = window_center - window_width / 2
            max_val = window_center + window_width / 2
            img = np.clip(img, min_val, max_val)  # 범위 내 값 유지
            img = (img - min_val) / (max_val - min_val) * 255  # 0~255 정규화
        else:
            # WL/WW 값이 없으면 기본적으로 min-max 정규화
            img = (img - img.min()) / (img.max() - img.min()) * 255

        Image.fromarray(img.astype(np.uint8)).save(folder_path+os.path.splitext(os.path.basename(dicom_file))[0].split('_')[-1]+'.png')

MRI

In [3]:


save_class_list1=['DWI']
class_list=['0100.Ischemic Stroke','0300.Stroke Control Group']
save_class_list=['Ischemic','Normal']
data_dir='../../data/stroke_2023/**/1.원천데이터/'
save_dir='../../data/stroke_mri1/'
mask_png_list=glob('../../data/stroke_mri/label/DWI/*')

In [ ]:
mask_png_folder_list=[os.path.basename(f) for f in mask_png_list]
i=6

nii_img=nib.load(nii_list[i])
nii_data=nii_img.get_fdata()
nii_data=nii_data.astype(np.float32)
folder_name=os.path.splitext(os.path.splitext(os.path.basename(nii_list[i]))[0])[0]
if folder_name in mask_png_folder_list:
    tumor_label=True
    index=mask_png_folder_list.index(folder_name)
    tumor_mask_path=mask_png_list[index]
    tumor_mask_list=glob(tumor_mask_path+'/*.png')
else:
    tumor_label=False
for j in range(len(nii_data)):
    slice_image = ((nii_data[j] - np.min(nii_data[j])) / (np.max(nii_data[j]) - np.min(nii_data[j])) * 255).astype(np.uint8)
    blured_image = cv2.GaussianBlur(slice_image, (5, 5), 0)
    mask = ((blured_image>np.percentile(blured_image, 50+j/2))*255).astype(np.uint8)
    mask=cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    mask=generate_largest_contour_mask_from_array(mask)
    if tumor_label:
        tumor_mask_list=sorted(tumor_mask_list)
        Valid_section1=len(tumor_mask_list)-64
        tumor_mask=cv2.imread(tumor_mask_list[Valid_section1+j])
        tumor_mask=cv2.cvtColor(tumor_mask, cv2.COLOR_BGR2GRAY)
        tumor_mask=cv2.resize(tumor_mask, (128, 128))
        tumor_index=np.where(tumor_mask>0)
        mask[tumor_index]=2
    
    final_mask_volume[j] = mask

# NIfTI로 저장
affine = np.eye(4)  # 단순 저장용 단위 행렬, 필요 시 원본에서 복사도 가능
nifti_img = nib.Nifti1Image(final_mask_volume, affine)
nib.save(nifti_img, f"{folder_name}_mask.nii.gz")